In [ ]:
using Pkg
# Pkg.add("Plots")
# Pkg.add("Interpolations")
# Pkg.add("QuadGK")
# Pkg.add("Unitful")
# Pkg.add("UnitfulAstro")
# Pkg.add("Random")
# Pkg.add("SpecialFunctions")
# Pkg.add("Roots")
# Pkg.add("Trapz")
# Pkg.add("PhysicalConstants")
# Pkg.add("Cosmology")
# Pkg.add("DataInterpolations")

In [41]:
using Plots 
using Interpolations
using QuadGK
using Unitful, UnitfulAstro
using Random
using SpecialFunctions
using Roots
using Trapz
using PhysicalConstants
using Cosmology
import DataInterpolations 

In [42]:
import PhysicalConstants.CODATA2018 as constants
const M_sun = 1.98847e30u"kg"
const T_cmb =  2.725 * u"K"
const P_e_factor = constants.σ_e / (constants.m_e * constants.c_0^2)

8.125531675591423e-16 s^2 kg^-1

In [43]:
abstract type AbstractProfileWorkspace{T} end
abstract type AbstractProfile{T} end
abstract type AbstractGNFW{T} <: AbstractProfile{T} end
abstract type AbstractInterpolatorProfile{T} <: AbstractProfile{T} end

In [72]:
function get_cosmology(::Type{T}; h=0.69,
                   Neff=3.04,
                   OmegaK=0.0,
                   OmegaM=0.29,
                   OmegaR=nothing,
                   Tcmb=2.7255,
                   w0=-1,
                   wa=0) where T

    if OmegaR === nothing
        OmegaG = 4.48131e-7*Tcmb^4/h^2
        OmegaN = Neff*OmegaG*(7/8)*(4/11)^(4/3)
        OmegaR = OmegaG + OmegaN
    end

    OmegaL = 1 - OmegaK - OmegaM - OmegaR

    if !(w0 == -1 && wa == 0)
        return Cosmology.WCDM{T}(h, OmegaK, OmegaL, OmegaM, OmegaR, w0, wa)
    end

    if OmegaK < 0
        return Cosmology.ClosedLCDM{T}(h, OmegaK, OmegaL, OmegaM, OmegaR)
    elseif OmegaK > 0
        return Cosmology.OpenLCDM{T}(h, OmegaK, OmegaL, OmegaM, OmegaR)
    else
        return Cosmology.FlatLCDM{T}(h, OmegaL, OmegaM, OmegaR)
    end
end
get_cosmology(; h=0.69, Neff=3.04, OmegaK=0.0, OmegaM=0.29, OmegaR=nothing, Tcmb=2.7255, 
    w0=-1, wa=0) = get_cosmology(Float32; h=h, Neff=Neff, OmegaK=OmegaK, 
        OmegaM=OmegaM, OmegaR=OmegaR, Tcmb=Tcmb, w0=w0, wa=wa)


function ρ_crit(model, z)
    H_z = H(model.cosmo, z)
    return uconvert(u"kg/m^3", 3H_z^2 / (8π * constants.G))
end

function R_Δ(model, M_Δ, z, Δ=200)
    return ∛(M_Δ / (4π/3 * Δ * ρ_crit(model, z)))
end

function angular_size(model::AbstractProfile{T}, physical_size, z) where T
    d_A = angular_diameter_dist(model.cosmo, z)

    # convert both to the same units and strip units for atan
    phys_siz_unitless = T(ustrip(uconvert(unit(d_A), physical_size)))
    d_A_unitless = T(ustrip(d_A))
    return atan(phys_siz_unitless, d_A_unitless)
end

function build_z2r_interpolator(min_z::T, max_z::T,
    cosmo::Cosmology.AbstractCosmology; n_bins=2000) where T

    zrange = LinRange(min_z, max_z, n_bins)
    rrange = zero(zrange)
    for i in 1:n_bins
        rrange[i] = ustrip(T, u"Mpc",
            Cosmology.comoving_radial_dist(u"Mpc", cosmo, zrange[i]))
    end
    z2r = DataInterpolations.LinearInterpolation(rrange, zrange);
    return z2r
end

function δαz2gnom(cosmo::Cosmology.AbstractCosmology, z2r::DataInterpolations.LinearInterpolation,
    δ, α, z; δ₀ = 0., α₀ = 0., ΔZ = 0.05, NZ::Int = 100)
    χ₀ = z2r(z)
    Dₐ = angular_diameter_dist(cosmo,z)
    χ₁, χ₂ = z2r(z - ΔZ), z2r(z + ΔZ)
    Z̃ = collect(range(χ₁ - χ₀, χ₂ - χ₀, length=NZ))
    C̃ = cos(δ₀)*cos(δ)*cos(α - α₀) + sin(δ)*sin(δ₀)
    X̃ = Dₐ*(cos(δ)*sin(α - α₀))/C̃
    Ỹ = Dₐ*(sin(δ₀)*cos(δ)*cos(α - α₀) - cos(δ₀)*sin(δ))/C̃
    return X̃, Ỹ, Z̃
end

function rotation_matrix(ϕ, θ, ψ)
    R_z1 = [cos(ϕ) -sin(ϕ) 0;
            sin(ϕ)  cos(ϕ) 0;
            0       0      1]
    R_x = [1  0         0;
           0  cos(θ) -sin(θ);
           0  sin(θ)  cos(θ)]
    R_z2 = [cos(ψ) -sin(ψ) 0;
            sin(ψ)  cos(ψ) 0;
            0       0      1]
    return R_z1 * R_x * R_z2
end

function get_coords(δ, α, z, δ₀, α₀, 
        cosmo::Cosmology.AbstractCosmology; 
        ϕ = 0, θ = 0, ψ = 0, ϵ₁ = 0, ϵ₂ = 0,
        z2r::Union{DataInterpolations.LinearInterpolation, Nothing} = nothing)
    R_rot = rotation_matrix(ϕ, θ, ψ)
    if z2r === nothing
        z2r = build_z2r_interpolator(1e-10, 5., cosmo)
    end
    x,y,z = δαz2gnom(cosmo, z2r, δ, α, z, δ₀, α₀)

    v⃗ = coords = [vec(x); vec(y); vec(z)]
    v⃗′ = R_rot * v⃗
    x′ = v⃗[1, :]
    y′ = v⃗[2, :]
    z′ = v⃗[3, :]
    R = @. sqrt(x′^2 + y′^2/(1 - ϵ₁)^2 + z′^2/(1 - ϵ₂)^2)
    return R
end

get_coords (generic function with 1 method)

In [73]:
δ = [deg2rad(v) for v in collect(range(150., 170., length = 10))]
α = [deg2rad(v) for v in collect(range(-5., -1., length = 10))]
δ₀ = deg2rad(155.)
α₀ = deg2rad(-2.)
z = 0.35
cosmo = cosmology()
z2r_interpolator = build_z2r_interpolator(1e-5, 5., cosmo)
get_coords(δ, α, z, δ₀, α₀,cosmo; ϕ = deg2rad(45), θ = 0., ψ = 0., z2r = z2r_interpolator)

LoadError: MethodError: no method matching δαz2gnom(::Cosmology.FlatLCDM{Float64}, ::DataInterpolations.LinearInterpolation{Vector{Float64}, LinRange{Float64, Int64}, Vector{Float64}, Vector{Float64}, Float64}, ::Vector{Float64}, ::Vector{Float64}, ::Float64, ::Float64, ::Float64)
The function `δαz2gnom` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  δαz2gnom(::Cosmology.AbstractCosmology, ::DataInterpolations.LinearInterpolation, ::Any, ::Any, ::Any; δ₀, α₀, ΔZ, NZ)
[0m[90m   @[39m [32mMain[39m [90m[4mIn[72]:66[24m[39m
[0m  δαz2gnom(::Cosmology.AbstractCosmology, ::DataInterpolations.LinearInterpolation, ::T, ::T, [91m::T[39m; δ₀, α₀, ΔZ, NZ) where T
[0m[90m   @[39m [32mMain[39m [90m[4mIn[66]:66[24m[39m
